# Data Analysis

# Notebook set-up

In [233]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sb
import io
import statistics as stat
import statsmodels.api as sm
from statsmodels.api import OLS
import random
import statsmodels.formula.api as smf
import scipy as sp
import scipy.stats
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from pandas.tools.plotting import scatter_matrix, radviz
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

sns.set_style('whitegrid')
sns.set_context('notebook')

# Data loading

In [234]:
# Load the data into a dataframe 
#can_master = pd.read_csv('can-1996-2016.csv')
#cmte_master =pd.read_csv('cm-1996-2014.csv') 
#opex=pd.read_csv('opex-2004-2014.csv')
#oth=pd.read_csv('oth-1996-2014.csv')
#exp=pd.read_csv('expresults_rev.csv')
#election_spend=pd.read_csv('catall.csv')
#results = pd.read_csv('candr.csv')
#catsort= pd.read_csv('catsort_rev.csv')
cost_vote = pd.read_csv('combined_all.csv')

In [235]:
cost_vote['total_votes']=cost_vote['total_votes'].fillna(0)
cost_vote['totalspend']=cost_vote['totalspend'].fillna(0)
cost_vote['incumbent']=cost_vote['incumbent'].fillna(0)
cost_vote['party']=cost_vote['party'].fillna('None')
cost_vote = cost_vote.drop(['costpervote'], axis=1)
cost_vote['totalspend']=pd.to_numeric(cost_vote['totalspend'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['totalspend'], errors = 'coerce').isnull()])
cost_vote['total_votes']=pd.to_numeric(cost_vote['total_votes'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['total_votes'], errors = 'coerce').isnull()])
cost_vote['cycle']=pd.to_numeric(cost_vote['cycle'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['cycle'], errors = 'coerce').isnull()])
cost_vote = cost_vote[cost_vote.total_votes != 0]
cost_vote.loc[cost_vote['totalspend'] == 0]
cost_vote = cost_vote.rename(columns = {'ge_winner_indicator':'winner'})
cost_vote['costvote'] = cost_vote['totalspend']/cost_vote['total_votes']

Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]
Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]
Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]


In [236]:
cost_vote['cycle'] = cost_vote['cycle'].astype(int).astype('str')

In [237]:
cost_vote['party'] = cost_vote['party'].astype('str')
cost_vote = cost_vote.replace("DEM","D")
cost_vote = cost_vote.replace("DEM ","D")
cost_vote = cost_vote.replace("REP","R")
cost_vote = cost_vote.replace("REP ","R")
R = cost_vote[cost_vote['party'] == "R"]
D = cost_vote[cost_vote['party'] == "D"]

In [238]:
testframes = [R, D]

Gen = pd.concat(testframes)

In [241]:
Gen = Gen.replace("W (Runoff)","0")
Gen = Gen.replace("WW","0")
Gen = Gen.replace("Dd","0")

In [243]:
Gen['race'] = Gen['cand_id'].astype(str).str[0]

In [244]:
Gen = Gen[Gen['race'] == "H"]

In [245]:
Gen.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote,race
1,H0AK01038,2010,1569.23,88.93,0.00,0.00,0.0,0.00,0.00,0.0,...,R,24709.0,0.0,0.0,0.0,0,24709.0,169777.74,6.871089,H
3,H0AL01030,2010,7054.34,5233.32,668.95,27082.28,1175.0,1295.21,583.75,0.0,...,R,18725.0,0.0,0.0,0.0,0,18725.0,44377.00,2.369933,H
4,H0AL01030,2012,3480.00,27.72,381.09,7257.71,0.0,0.00,56.58,0.0,...,R,3854.0,0.0,0.0,0.0,0,3854.0,11203.10,2.906876,H
5,H0AL02087,2012,11727.82,0.00,16964.09,0.00,0.0,0.00,0.00,0.0,...,R,0.0,0.0,180591.0,0.0,1,180591.0,694953.98,3.848220,H
6,H0AL02087,2014,12132.25,0.00,16866.78,0.00,0.0,0.00,0.00,0.0,...,R,0.0,0.0,113103.0,0.0,1,113103.0,570473.48,5.043840,H


## Scores for Democrats (blue font unintentional) 

In [248]:
Winner = Gen[Gen['winner'].isin(['1'])]

In [249]:
Loser = Gen[Gen['winner'].isin(['0'])]

In [250]:
Dem = Gen[Gen['party'].isin(['D'])]

In [252]:
DemW = Dem[Dem['winner'].isin(['1'])] #table of Democrats who won


In [253]:
DemWInc = DemW[DemW['incumbent'].isin(['1'])] #table of Democrat Incumbent who won

In [193]:
DemWC = DemW[DemW['incumbent'].isin(['0'])] #table of Democrat challengers who won
DemWC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,165.0,1.815879e+05,2.921361e+05,0.0,0.000000e+00,3.063620e+04,2.836497e+05,1.897445e+06
c2,165.0,5.538201e+03,9.794140e+03,0.0,0.000000e+00,0.000000e+00,6.972830e+03,5.004063e+04
c3,165.0,3.662263e+04,7.297284e+04,0.0,0.000000e+00,3.874500e+03,4.629797e+04,5.949072e+05
c4,165.0,2.888834e+05,5.598700e+05,0.0,0.000000e+00,0.000000e+00,2.690044e+05,2.826205e+06
c5,165.0,2.640633e+04,6.238465e+04,0.0,0.000000e+00,0.000000e+00,3.821250e+04,6.570344e+05
c6,165.0,5.654358e+04,1.990169e+05,0.0,0.000000e+00,0.000000e+00,4.218268e+04,2.290329e+06
c7,165.0,5.287069e+03,1.068085e+04,0.0,0.000000e+00,0.000000e+00,5.691860e+03,6.407480e+04
c8,165.0,6.134182e+02,7.862755e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.010000e+05
c9,165.0,1.716952e+02,1.779977e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.218561e+04
c10,165.0,1.774302e+02,7.976431e+02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,8.313270e+03


In [254]:
DemL = Dem[Dem['winner'].isin(['0'])] #table of Democrats who lost

In [255]:
DemLInc = DemL[DemL['incumbent'].isin(['1'])] #table of Democrat Incumbent who lost

In [256]:
DemLC = DemL[DemL['incumbent'].isin(['0'])] #table of Democrat Incumbent who lost

In [257]:
Rep = Gen[Gen['party'].isin(['R'])]

In [258]:
RepW = Rep[Rep['winner'].isin(['1'])]

In [259]:
RepWInc = RepW[RepW['incumbent'].isin(['1'])]

In [260]:
RepWC = Rep[Rep['incumbent'].isin(['0'])]

In [261]:
RepL = Rep[Rep['winner'].isin(['0'])]

In [262]:
RepLInc = RepL[RepL['incumbent'].isin(['1'])]

In [263]:
RepLC = RepL[RepL['incumbent'].isin(['0'])]

In [264]:
DemW['total_votes'].sum()

215183428.0

In [265]:
formula = 'total_votes ~ totalspend'
model = smf.ols(formula, data=Winner)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_votes   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     9.109
Date:                Fri, 08 Sep 2017   Prob (F-statistic):            0.00257
Time:                        22:31:01   Log-Likelihood:                -30768.
No. Observations:                2469   AIC:                         6.154e+04
Df Residuals:                    2467   BIC:                         6.155e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.885e+05   1861.919    101.222      0.000      1.85e+05  1.92e+05
totalspend    -0.0037      0.001     -3.018      0.003        -0.006    -0.001
==============================================================================
Omnibus:                       17.352   Durbin-Watson:                   1.474
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.243
Skew:                           0.136   Prob(JB):                     4.02e-05
Kurtosis:                       3.351   Cond. No.                     2.27e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.27e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [266]:
formula = 'total_votes ~ totalspend'
model = smf.ols(formula, data=Loser)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_votes   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     1445.
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          2.23e-284
Time:                        22:31:03   Log-Likelihood:                -75201.
No. Observations:                6176   AIC:                         1.504e+05
Df Residuals:                    6174   BIC:                         1.504e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   3.365e+04    648.139     51.924      0.000      3.24e+04  3.49e+04
totalspend     0.0329      0.001     38.019      0.000         0.031     0.035
==============================================================================
Omnibus:                      931.588   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1630.083
Skew:                           0.982   Prob(JB):                         0.00
Kurtosis:                       4.573   Cond. No.                     8.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [267]:
formula = 'total_votes ~ c1+c2+c3+c4+c5+c6+c7+c8+c9+c10+c11+c12+c13'
model = smf.ols(formula, data=DemW)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_votes   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.006
Date:                Fri, 08 Sep 2017   Prob (F-statistic):              0.443
Time:                        22:31:06   Log-Likelihood:                -15029.
No. Observations:                1195   AIC:                         3.009e+04
Df Residuals:                    1181   BIC:                         3.016e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.869e+05   3354.502     55.715      0.000       1.8e+05  1.93e+05
c1            -0.0053      0.014     -0.376      0.707        -0.033     0.022
c2            -0.0437      0.160     -0.273      0.785        -0.357     0.270
c3            -0.0240      0.031     -0.782      0.435        -0.084     0.036
c4            -0.0109      0.010     -1.095      0.274        -0.030     0.009
c5             0.0215      0.099      0.218      0.828        -0.172     0.215
c6            -0.0268      0.027     -0.997      0.319        -0.080     0.026
c7            -0.0341      0.067     -0.511      0.610        -0.165     0.097
c8             0.1679      0.337      0.498      0.619        -0.494     0.830
c9            -1.8272      1.249     -1.463      0.144        -4.278     0.624
c10           -1.1632      4.650     -0.250      0.803       -10.287     7.960
c11            0.1030      0.155      0.664      0.507        -0.202     0.408
c12           -0.0771      0.360     -0.214      0.830        -0.783     0.629
c13           -0.0046      0.003     -1.813      0.070        -0.009     0.000
==============================================================================
Omnibus:                        8.201   Durbin-Watson:                   1.403
Prob(Omnibus):                  0.017   Jarque-Bera (JB):                8.165
Skew:                           0.200   Prob(JB):                       0.0169
Kurtosis:                       3.061   Cond. No.                     1.92e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.92e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [269]:
formula = 'total_votes ~ c1+c2+c3+c4+c5+c6+c7+c8+c9+c12+c13'
model = smf.ols(formula, data=Loser)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_votes   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     148.3
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          4.55e-304
Time:                        22:31:20   Log-Likelihood:                -75126.
No. Observations:                6176   AIC:                         1.503e+05
Df Residuals:                    6164   BIC:                         1.504e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   3.263e+04    649.464     50.237      0.000      3.14e+04  3.39e+04
c1             0.0367      0.004      9.290      0.000         0.029     0.044
c2             0.6490      0.100      6.476      0.000         0.453     0.845
c3            -0.0017      0.005     -0.312      0.755        -0.012     0.009
c4             0.0049      0.004      1.180      0.238        -0.003     0.013
c5             0.3914      0.069      5.692      0.000         0.257     0.526
c6             0.0273      0.016      1.674      0.094        -0.005     0.059
c7             0.0985      0.077      1.272      0.204        -0.053     0.250
c8            -0.7950      1.459     -0.545      0.586        -3.654     2.064
c9            -0.5926      0.765     -0.775      0.438        -2.092     0.907
c12            0.0599      0.346      0.173      0.862        -0.618     0.738
c13            0.0372      0.001     31.697      0.000         0.035     0.039
==============================================================================
Omnibus:                      907.204   Durbin-Watson:                   1.624
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1413.943
Skew:                           1.031   Prob(JB):                    9.25e-308
Kurtosis:                       4.117   Cond. No.                     5.80e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Simple Regression for Winners and Losers

In [270]:
Total_VoteW = Winner['total_votes']
Total_SpendW = Winner['totalspend']

def reg_m(Total_VoteW,Total_SpendW):
    Total_SpendW=np.array(Total_SpendW).T
    Total_SpendW=sm.add_constant(Total_SpendW)
    model=sm.OLS(endog=Total_VoteW,exog=Total_SpendW)
    results=model.fit()
    return results

print (reg_m(Total_VoteW,Total_SpendW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     9.109
Date:                Fri, 08 Sep 2017   Prob (F-statistic):            0.00257
Time:                        22:31:26   Log-Likelihood:                -30768.
No. Observations:                2469   AIC:                         6.154e+04
Df Residuals:                    2467   BIC:                         6.155e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.885e+05   1861.919    101.222      0.0

In [272]:
Total_VoteL = Loser['total_votes']
Total_SpendL = Loser['totalspend']

def reg_m(Total_VoteL,Total_SpendL):
    Total_SpendL=np.array(Total_SpendL).T
    Total_SpendL=sm.add_constant(Total_SpendL)
    model=sm.OLS(endog=Total_VoteL,exog=Total_SpendL)
    results=model.fit()
    return results

print (reg_m(Total_VoteL,Total_SpendL).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     1445.
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          2.23e-284
Time:                        22:31:33   Log-Likelihood:                -75201.
No. Observations:                6176   AIC:                         1.504e+05
Df Residuals:                    6174   BIC:                         1.504e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       3.365e+04    648.139     51.924      0.0

Simple Regression for Democrats and Republicans who won the General Election

In [273]:
Total_VoteDW = DemW['total_votes']
Total_SpendDW = DemW['totalspend']

def reg_m(Total_VoteDW,Total_SpendDW):
    Total_SpendDW=np.array(Total_SpendDW).T
    Total_SpendDW=sm.add_constant(Total_SpendDW)
    model=sm.OLS(endog=Total_VoteDW,exog=Total_SpendDW)
    results=model.fit()
    return results

print (reg_m(Total_VoteDW,Total_SpendDW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     6.282
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0123
Time:                        22:31:38   Log-Likelihood:                -15033.
No. Observations:                1195   AIC:                         3.007e+04
Df Residuals:                    1193   BIC:                         3.008e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.861e+05   3138.012     59.291      0.0

In [274]:
Total_VoteDWInc = DemWInc['total_votes']
Total_SpendDWInc = DemWInc['totalspend']

def reg_m(Total_VoteDWInc,Total_SpendDWInc):
    Total_SpendDWInc=np.array(Total_SpendDWInc).T
    Total_SpendDWInc=sm.add_constant(Total_SpendDWInc)
    model=sm.OLS(endog=Total_VoteDWInc,exog=Total_SpendDWInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteDWInc,Total_SpendDWInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.120
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0135
Time:                        22:31:48   Log-Likelihood:                -12982.
No. Observations:                1030   AIC:                         2.597e+04
Df Residuals:                    1028   BIC:                         2.598e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         1.9e+05   3485.805     54.498      0.0

In [275]:
Total_VoteDWC = DemWC['total_votes']
Total_SpendDWC = DemWC['totalspend']

def reg_m(Total_VoteDWC,Total_SpendDWC):
    Total_SpendDWC=np.array(Total_SpendDWC).T
    Total_SpendDWC=sm.add_constant(Total_SpendDWC)
    model=sm.OLS(endog=Total_VoteDWC,exog=Total_SpendDWC)
    results=model.fit()
    return results

print (reg_m(Total_VoteDWC,Total_SpendDWC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     6.609
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0110
Time:                        22:31:51   Log-Likelihood:                -2029.0
No. Observations:                 165   AIC:                             4062.
Df Residuals:                     163   BIC:                             4068.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.426e+05   7759.672     18.375      0.0

In [276]:
Gen.to_csv('General.csv', index=False)

In [277]:
Total_VoteRW = RepW['total_votes']
Total_SpendRW = RepW['totalspend']

def reg_m(Total_VoteRW,Total_SpendRW):
    Total_SpendRW=np.array(Total_SpendRW).T
    Total_SpendRW=sm.add_constant(Total_SpendRW)
    model=sm.OLS(endog=Total_VoteRW,exog=Total_SpendRW)
    results=model.fit()
    return results

print (reg_m(Total_VoteRW,Total_SpendRW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     5.456
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0197
Time:                        22:31:53   Log-Likelihood:                -15685.
No. Observations:                1274   AIC:                         3.137e+04
Df Residuals:                    1272   BIC:                         3.138e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        1.92e+05   2188.976     87.724      0.0

In [278]:
Total_VoteRWInc = RepWInc['total_votes']
Total_SpendRWInc = RepWInc['totalspend']

def reg_m(Total_VoteRWInc,Total_SpendRWInc):
    Total_SpendRWInc=np.array(Total_SpendRWInc).T
    Total_SpendRWInc=sm.add_constant(Total_SpendRWInc)
    model=sm.OLS(endog=Total_VoteRWInc,exog=Total_SpendRWInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteRWInc,Total_SpendRWInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.473
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0627
Time:                        22:31:58   Log-Likelihood:                -12983.
No. Observations:                1057   AIC:                         2.597e+04
Df Residuals:                    1055   BIC:                         2.598e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.966e+05   2258.408     87.058      0.0

In [279]:
Total_VoteRWC = RepWC['total_votes']
Total_SpendRWC = RepWC['totalspend']

def reg_m(Total_VoteRWC,Total_SpendRWC):
    Total_SpendRWC=np.array(Total_SpendRWC).T
    Total_SpendRWC=sm.add_constant(Total_SpendRWC)
    model=sm.OLS(endog=Total_VoteRWC,exog=Total_SpendRWC)
    results=model.fit()
    return results

print (reg_m(Total_VoteRWC,Total_SpendRWC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     1456.
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          5.24e-266
Time:                        22:31:58   Log-Likelihood:                -41914.
No. Observations:                3450   AIC:                         8.383e+04
Df Residuals:                    3448   BIC:                         8.384e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.756e+04    857.845     32.133      0.0

In [280]:
Total_VoteDL = DemL['total_votes']
Total_SpendDL = DemL['totalspend']

def reg_m(Total_VoteDL,Total_SpendDL):
    Total_SpendDL=np.array(Total_SpendDL).T
    Total_SpendDL=sm.add_constant(Total_SpendDL)
    model=sm.OLS(endog=Total_VoteDL,exog=Total_SpendDL)
    results=model.fit()
    return results

print (reg_m(Total_VoteDL,Total_SpendDL).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     625.5
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          6.59e-125
Time:                        22:32:01   Log-Likelihood:                -34736.
No. Observations:                2834   AIC:                         6.948e+04
Df Residuals:                    2832   BIC:                         6.949e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       4.074e+04   1048.633     38.848      0.0

In [281]:
Total_VoteDLInc = DemLInc['total_votes']
Total_SpendDLInc = DemLInc['totalspend']

def reg_m(Total_VoteDLInc,Total_SpendDLInc):
    Total_SpendDLInc=np.array(Total_SpendDLInc).T
    Total_SpendDLINc=sm.add_constant(Total_SpendDLInc)
    model=sm.OLS(endog=Total_VoteDLInc,exog=Total_SpendDLInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteDLInc,Total_SpendDLInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                     205.5
Date:                Fri, 08 Sep 2017   Prob (F-statistic):           1.39e-29
Time:                        22:32:02   Log-Likelihood:                -1859.5
No. Observations:                 146   AIC:                             3721.
Df Residuals:                     145   BIC:                             3724.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             0.0447      0.003     14.336      0.0

In [282]:
Total_VoteDLC = DemLC['total_votes']
Total_SpendDLC = DemLC['totalspend']

def reg_m(Total_VoteDLC,Total_SpendDLC):
    Total_SpendDLC=np.array(Total_SpendDLC).T
    Total_SpendDLC=sm.add_constant(Total_SpendDLC)
    model=sm.OLS(endog=Total_VoteDLC,exog=Total_SpendDLC)
    results=model.fit()
    return results

print (reg_m(Total_VoteDLC,Total_SpendDLC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     512.9
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          4.66e-104
Time:                        22:32:03   Log-Likelihood:                -32874.
No. Observations:                2687   AIC:                         6.575e+04
Df Residuals:                    2685   BIC:                         6.576e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       3.859e+04   1044.634     36.942      0.0

In [283]:
Total_VoteRL = RepL['total_votes']
Total_SpendRL = RepL['totalspend']

def reg_m(Total_VoteRL,Total_SpendRL):
    Total_SpendRL=np.array(Total_SpendRL).T
    Total_SpendRL=sm.add_constant(Total_SpendRL)
    model=sm.OLS(endog=Total_VoteRL,exog=Total_SpendRL)
    results=model.fit()
    return results

print (reg_m(Total_VoteRL,Total_SpendRL).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     871.5
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          2.10e-170
Time:                        22:32:04   Log-Likelihood:                -40332.
No. Observations:                3342   AIC:                         8.067e+04
Df Residuals:                    3340   BIC:                         8.068e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.763e+04    784.247     35.229      0.0

In [284]:
Total_VoteRLInc = RepLInc['total_votes']
Total_SpendRLInc = RepLInc['totalspend']

def reg_m(Total_VoteRLInc,Total_SpendRLInc):
    Total_SpendRLInc=np.array(Total_SpendRLInc).T
    Total_SpendRLInc=sm.add_constant(Total_SpendRLInc)
    model=sm.OLS(endog=Total_VoteRLInc,exog=Total_SpendRLInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteRLInc,Total_SpendRLInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     2.667
Date:                Fri, 08 Sep 2017   Prob (F-statistic):              0.105
Time:                        22:32:05   Log-Likelihood:                -1341.8
No. Observations:                 108   AIC:                             2688.
Df Residuals:                     106   BIC:                             2693.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.129e+05   8581.655     13.154      0.0

In [285]:
Total_VoteRLC = RepLC['total_votes']
Total_SpendRLC = RepLC['totalspend']

def reg_m(Total_VoteRLC,Total_SpendRLC):
    Total_SpendRLC=np.array(Total_SpendRLC).T
    Total_SpendRLC=sm.add_constant(Total_SpendRLC)
    model=sm.OLS(endog=Total_VoteRLC,exog=Total_SpendRLC)
    results=model.fit()
    return results

print (reg_m(Total_VoteRLC,Total_SpendRLC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     694.1
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          9.77e-139
Time:                        22:32:08   Log-Likelihood:                -38835.
No. Observations:                3233   AIC:                         7.767e+04
Df Residuals:                    3231   BIC:                         7.769e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.543e+04    760.659     33.430      0.0

Multi-linear regressions

In [286]:
Total_VoteW = Winner['total_votes']
Total_SpendW = [Winner['c1'],Winner['c2'],Winner['c3'],Winner['c4'],Winner['c5'],Winner['c6'],Winner['c7'],Winner['c8'],Winner['c9'],Winner['c10'],Winner['c11'],Winner['c12'],Winner['c13']]

def reg_m(Total_VoteW,Total_SpendW):
    Total_SpendW=np.array(Total_SpendW).T
    Total_SpendW=sm.add_constant(Total_SpendW)
    model=sm.OLS(endog=Total_VoteW,exog=Total_SpendW)
    results=model.fit()
    return results

print (reg_m(Total_VoteW,Total_SpendW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.059
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0138
Time:                        22:32:09   Log-Likelihood:                -30760.
No. Observations:                2469   AIC:                         6.155e+04
Df Residuals:                    2455   BIC:                         6.163e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.885e+05   1913.003     98.534      0.0

In [288]:
Total_VoteL = Loser['total_votes']
Total_SpendL = [Loser['c1'],Loser['c2'],Loser['c3'],Loser['c4'],Loser['c5'],Loser['c6'],Loser['c7'],Loser['c8'],Loser['c9'],Loser['c10'],Loser['c11'],Loser['c12'],Loser['c13']]

def reg_m(Total_VoteL,Total_SpendL):
    Total_SpendL=np.array(Total_SpendL).T
    Total_SpendL=sm.add_constant(Total_SpendL)
    model=sm.OLS(endog=Total_VoteL,exog=Total_SpendL)
    results=model.fit()
    return results

print (reg_m(Total_VoteL,Total_SpendL).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     127.9
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          2.33e-307
Time:                        22:32:26   Log-Likelihood:                -75113.
No. Observations:                6176   AIC:                         1.503e+05
Df Residuals:                    6162   BIC:                         1.503e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       3.254e+04    648.583     50.169      0.0

In [289]:
Total_VoteD = Dem['total_votes']
Total_SpendD = [Dem['c1'],Dem['c2'],Dem['c3'],Dem['c4'],Dem['c5'],Dem['c6'],Dem['c7'],Dem['c8'],Dem['c9'],Dem['c10'],Dem['c11'],Dem['c12'],Dem['c13']]

def reg_m(Total_VoteD,Total_SpendD):
    Total_SpendD=np.array(Total_SpendD).T
    Total_SpendD=sm.add_constant(Total_SpendD)
    model=sm.OLS(endog=Total_VoteD,exog=Total_SpendD)
    results=model.fit()
    return results

print (reg_m(Total_VoteD,Total_SpendD).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     88.54
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          2.30e-208
Time:                        22:32:29   Log-Likelihood:                -50919.
No. Observations:                4029   AIC:                         1.019e+05
Df Residuals:                    4015   BIC:                         1.020e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       6.329e+04   1419.468     44.585      0.0

In [290]:
Total_VoteR = Rep['total_votes']
Total_SpendR = [Rep['c1'],Rep['c2'],Rep['c3'],Rep['c4'],Rep['c5'],Rep['c6'],Rep['c7'],Rep['c8'],Rep['c9'],Rep['c10'],Rep['c11'],Rep['c12'],Rep['c13']]

def reg_m(Total_VoteR,Total_SpendR):
    Total_SpendR=np.array(Total_SpendR).T
    Total_SpendR=sm.add_constant(Total_SpendR)
    model=sm.OLS(endog=Total_VoteR,exog=Total_SpendR)
    results=model.fit()
    return results

print (reg_m(Total_VoteR,Total_SpendR).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     129.2
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          1.12e-298
Time:                        22:32:31   Log-Likelihood:                -58170.
No. Observations:                4616   AIC:                         1.164e+05
Df Residuals:                    4602   BIC:                         1.165e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.289e+04   1237.806     42.731      0.0

In [294]:
Total_VoteDW = DemW['total_votes']
Total_SpendDW = [DemW['c1'],DemW['c2'],DemW['c3'],DemW['c4'],DemW['c5'],DemW['c6'],DemW['c7'],DemW['c8'],DemW['c9'],DemW['c10'],DemW['c11'],DemW['c12'],DemW['c13']]

def reg_m(Total_VoteDW,Total_SpendDW):
    Total_SpendDW=np.array(Total_SpendDW).T
    Total_SpendDW=sm.add_constant(Total_SpendDW)
    model=sm.OLS(endog=Total_VoteDW,exog=Total_SpendDW)
    results=model.fit()
    return results

print (reg_m(Total_VoteDW,Total_SpendDW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.006
Date:                Fri, 08 Sep 2017   Prob (F-statistic):              0.443
Time:                        22:34:32   Log-Likelihood:                -15029.
No. Observations:                1195   AIC:                         3.009e+04
Df Residuals:                    1181   BIC:                         3.016e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.869e+05   3354.502     55.715      0.0

In [292]:
Total_VoteRW = RepW['total_votes']
Total_SpendRW = [RepW['c1'],RepW['c2'],RepW['c3'],RepW['c4'],RepW['c5'],RepW['c6'],RepW['c7'],RepW['c8'],RepW['c9'],RepW['c10'],RepW['c11'],RepW['c12'],RepW['c13']]

def reg_m(Total_VoteRW,Total_SpendRW):
    Total_SpendRW=np.array(Total_SpendRW).T
    Total_SpendRW=sm.add_constant(Total_SpendRW)
    model=sm.OLS(endog=Total_VoteRW,exog=Total_SpendRW)
    results=model.fit()
    return results

print (reg_m(Total_VoteRW,Total_SpendRW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.059
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0141
Time:                        22:33:12   Log-Likelihood:                -15674.
No. Observations:                1274   AIC:                         3.138e+04
Df Residuals:                    1260   BIC:                         3.145e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.926e+05   2278.159     84.539      0.0

In [296]:
#Democrat incumbents who won in the general election
Total_VoteDWInc = DemWInc['total_votes']
Total_SpendDWInc = [DemWInc['c1'],DemWInc['c2'],DemWInc['c3'],DemWInc['c4'],DemWInc['c5'],DemWInc['c6'],DemWInc['c7'],DemWInc['c8'],DemWInc['c9'],DemWInc['c10'],DemWInc['c11'],DemWInc['c12'],DemWInc['c13']]

def reg_m(Total_VoteDWInc,Total_SpendDWInc):
    Total_SpendDWInc=np.array(Total_SpendDWInc).T
    Total_SpendDWInc=sm.add_constant(Total_SpendDWInc)
    model=sm.OLS(endog=Total_VoteDWInc,exog=Total_SpendDWInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteDWInc,Total_SpendDWInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.064
Date:                Fri, 08 Sep 2017   Prob (F-statistic):              0.387
Time:                        22:35:53   Log-Likelihood:                -12978.
No. Observations:                1030   AIC:                         2.598e+04
Df Residuals:                    1016   BIC:                         2.605e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.916e+05   3703.850     51.718      0.0

In [297]:
#Republican Incumbents who won in the general election
Total_VoteRWInc = RepWInc['total_votes']
Total_SpendRWInc = [RepWInc['c1'],RepWInc['c2'],RepWInc['c3'],RepWInc['c4'],RepWInc['c5'],RepWInc['c6'],RepWInc['c7'],RepWInc['c8'],RepWInc['c9'],RepWInc['c10'],RepWInc['c11'],RepWInc['c12'],RepWInc['c13']]

def reg_m(Total_VoteRWInc,Total_SpendRWInc):
    Total_SpendRWInc=np.array(Total_SpendRWInc).T
    Total_SpendRWInc=sm.add_constant(Total_SpendRWInc)
    model=sm.OLS(endog=Total_VoteRWInc,exog=Total_SpendRWInc)
    results=model.fit()
    return results

print (reg_m(Total_VoteRWInc,Total_SpendRWInc).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.066
Date:                Fri, 08 Sep 2017   Prob (F-statistic):              0.385
Time:                        22:35:54   Log-Likelihood:                -12978.
No. Observations:                1057   AIC:                         2.598e+04
Df Residuals:                    1043   BIC:                         2.605e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.974e+05   2374.261     83.136      0.0

In [298]:
#Democrat challengers who won in the general election
Total_VoteDWC = DemWC['total_votes']
Total_SpendDWC = [DemWC['c1'],DemWC['c2'],DemWC['c3'],DemWC['c4'],DemWC['c5'],DemWC['c6'],DemWC['c7'],DemWC['c8'],DemWC['c9'],DemWC['c10'],DemWC['c11'],DemWC['c12'],DemWC['c13']]

def reg_m(Total_VoteDWC,Total_SpendDWC):
    Total_SpendDWC=np.array(Total_SpendDWC).T
    Total_SpendDWC=sm.add_constant(Total_SpendDWC)
    model=sm.OLS(endog=Total_VoteDWC,exog=Total_SpendDWC)
    results=model.fit()
    return results

print (reg_m(Total_VoteDWC,Total_SpendDWC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     1.696
Date:                Fri, 08 Sep 2017   Prob (F-statistic):             0.0669
Time:                        22:35:56   Log-Likelihood:                -2021.1
No. Observations:                 165   AIC:                             4070.
Df Residuals:                     151   BIC:                             4114.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.441e+05   8431.803     17.091      0.0

In [299]:
#Republican challengers who won in the general election
Total_VoteRWC = RepWC['total_votes']
Total_SpendRWC = [RepWC['c1'],RepWC['c2'],RepWC['c3'],RepWC['c4'],RepWC['c5'],RepWC['c6'],RepWC['c7'],RepWC['c8'],RepWC['c9'],RepWC['c10'],RepWC['c11'],RepWC['c12'],RepWC['c13']]

def reg_m(Total_VoteRWC,Total_SpendRWC):
    Total_SpendRWC=np.array(Total_SpendRWC).T
    Total_SpendRWC=sm.add_constant(Total_SpendRWC)
    model=sm.OLS(endog=Total_VoteRWC,exog=Total_SpendRWC)
    results=model.fit()
    return results

print (reg_m(Total_VoteRWC,Total_SpendRWC).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     128.8
Date:                Fri, 08 Sep 2017   Prob (F-statistic):          3.08e-284
Time:                        22:35:58   Log-Likelihood:                -41837.
No. Observations:                3450   AIC:                         8.370e+04
Df Residuals:                    3436   BIC:                         8.379e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.662e+04    849.845     31.325      0.0

Confidence Interval for General Votes

In [300]:
Total_Vote = Gen['total_votes']

def mean_confidence_interval(Total_Vote, confidence=0.95):
    a = 1.0*np.array(Total_Vote)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_Vote, confidence=0.95))

(83482.144823597453, 81701.697395669762, 85262.592251525144)


In [301]:
costvote = Gen['costvote']

def mean_confidence_interval(costvote, confidence=0.95):
    a = 1.0*np.array(costvote)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvote, confidence=0.95))

(10.446166968143295, 9.5668091478130375, 11.325524788473553)


Confidence Interval for General Election Winners and Losers

In [302]:
Total_VoteW = Winner['total_votes']

def mean_confidence_interval(Total_VoteW, confidence=0.95):
    a = 1.0*np.array(Total_VoteW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteW, confidence=0.95))

(184325.822600243, 181854.20740898463, 186797.43779150138)


In [303]:
costvoteW = Winner['costvote']

def mean_confidence_interval(costvoteW, confidence=0.95):
    a = 1.0*np.array(costvoteW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteW, confidence=0.95))

(7.1708926788598646, 6.8766993574201578, 7.4650860002995714)


In [304]:
Total_VoteL = Loser['total_votes']

def mean_confidence_interval(Total_VoteL, confidence=0.95):
    a = 1.0*np.array(Total_VoteL)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteL, confidence=0.95))

(43167.533354922278, 41865.586564483965, 44469.480145360591)


In [305]:
costvoteL = Loser['costvote']

def mean_confidence_interval(costvoteL, confidence=0.95):
    a = 1.0*np.array(costvoteL)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteL, confidence=0.95))

(11.755534231783319, 10.531687065390454, 12.979381398176184)


Confidence Interval for Democrats and Republicans in General Election

In [306]:
Total_VoteD = Dem['total_votes']

def mean_confidence_interval(Total_VoteD, confidence=0.95):
    a = 1.0*np.array(Total_VoteD)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteD, confidence=0.95))

(89618.017374038216, 87005.91963017665, 92230.115117899782)


In [307]:
costvoteD = Dem['costvote']

def mean_confidence_interval(costvoteD, confidence=0.95):
    a = 1.0*np.array(costvoteD)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteD, confidence=0.95))

(10.400559945134189, 8.7541908923424359, 12.046928997925942)


In [308]:
Total_VoteR = Rep['total_votes']

def mean_confidence_interval(Total_VoteR, confidence=0.95):
    a = 1.0*np.array(Total_VoteR)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteR, confidence=0.95))

(78126.548960138651, 75702.945864644091, 80550.152055633211)


In [309]:
costvoteR = Rep['costvote']

def mean_confidence_interval(costvoteR, confidence=0.95):
    a = 1.0*np.array(costvoteR)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteR, confidence=0.95))

(10.485974311233349, 9.6807289234839828, 11.291219698982715)


Confidence Intervals for Democrats and Republicans who who in General Election 

In [310]:
Total_VoteDW = DemW['total_votes']

def mean_confidence_interval(Total_VoteDW, confidence=0.95):
    a = 1.0*np.array(Total_VoteDW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteDW, confidence=0.95))

(180069.81422594143, 176066.61213110897, 184073.0163207739)


In [311]:
costvoteDW = DemW['costvote']

def mean_confidence_interval(costvoteDW, confidence=0.95):
    a = 1.0*np.array(costvoteDW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteDW, confidence=0.95))

(7.0124364088370026, 6.6069732320661476, 7.4178995856078576)


In [312]:
Total_VoteRW = RepW['total_votes']

def mean_confidence_interval(Total_VoteRW, confidence=0.95):
    a = 1.0*np.array(Total_VoteRW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteRW, confidence=0.95))

(188317.91836734695, 185355.34312548547, 191280.49360920844)


In [313]:
costvoteRW = RepW['costvote']

def mean_confidence_interval(costvoteRW, confidence=0.95):
    a = 1.0*np.array(costvoteRW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteRW, confidence=0.95))

(7.3195231676175734, 6.8943980744068414, 7.7446482608283054)


Confidence Interval of Democrats and Republican Incumbents who won the General Election

In [314]:
Total_VoteDWInc = DemWInc['total_votes']

def mean_confidence_interval(Total_VoteDWInc, confidence=0.95):
    a = 1.0*np.array(Total_VoteDWInc)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteDWInc, confidence=0.95))

(183375.10097087378, 178955.59196773914, 187794.60997400843)


In [315]:
costvoteDWInc = DemWInc['costvote']

def mean_confidence_interval(costvoteDWInc, confidence=0.95):
    a = 1.0*np.array(costvoteDWInc)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteDWInc, confidence=0.95))

(6.1994589242722391, 5.7943148208196664, 6.6046030277248118)


In [316]:
Total_VoteRWInc = RepWInc['total_votes']

def mean_confidence_interval(Total_VoteRWInc, confidence=0.95):
    a = 1.0*np.array(Total_VoteRWInc)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteRWInc, confidence=0.95))

(193660.6830652791, 190498.86345901378, 196822.50267154441)


In [317]:
costvoteRWInc = RepWInc['costvote']

def mean_confidence_interval(costvoteRWInc, confidence=0.95):
    a = 1.0*np.array(costvoteRWInc)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteRWInc, confidence=0.95))

(6.6228137177639201, 6.1849022513500271, 7.0607251841778131)


Confidence Intervals for Democrats and Republicans challengers who won the General Election

In [318]:
Total_VoteDWC = DemWC['total_votes']

def mean_confidence_interval(Total_VoteDWC, confidence=0.95):
    a = 1.0*np.array(Total_VoteDWC)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteDWC, confidence=0.95))

(159436.81212121213, 151098.67221591758, 167774.95202650668)


In [319]:
costvoteDWC = DemWC['costvote']

def mean_confidence_interval(costvoteDWC, confidence=0.95):
    a = 1.0*np.array(costvoteDWC)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteDWC, confidence=0.95))

(12.087386767029166, 10.838334032095045, 13.336439501963287)


In [320]:
Total_VoteRWC = RepWC['total_votes']

def mean_confidence_interval(Total_VoteRWC, confidence=0.95):
    a = 1.0*np.array(Total_VoteRWC)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteRWC, confidence=0.95))

(41312.284057971017, 39492.302493211282, 43132.265622730753)


In [321]:
costvoteRWC = RepWC['costvote']

def mean_confidence_interval(costvoteRWC, confidence=0.95):
    a = 1.0*np.array(costvoteRWC)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteRWC, confidence=0.95))

(11.224644169122037, 10.181625991662983, 12.267662346581091)


# Scikit Learn Regression

In [326]:
feature_cols = ['c1', 'c2','c3', 'c4','c5','c6','c7','c8','c9','c10','c11','c12','c13']

# use a list to select a subset of the original DataFrame
X = RepW[feature_cols]

# print the first five rows
X.head()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13
5,11727.82,0.00,16964.09,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,666262.07
6,12132.25,0.00,16866.78,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,541474.45
14,54616.04,2343.79,71452.12,547321.04,27000.0,66857.04,9362.66,249.0,0.0,0.0,0.0,0.0,1626.96
15,54178.00,8154.95,121673.90,165131.27,17000.0,16138.18,9083.92,0.0,0.0,0.0,0.0,0.0,2321.95
16,15420.65,8540.32,91996.02,81481.66,15000.0,11833.66,2350.00,0.0,0.0,0.0,0.0,0.0,215.00


In [327]:
print (type(X))
print (X.shape)

<class 'pandas.core.frame.DataFrame'>
(1274, 13)


In [328]:
y = RepW['total_votes']
y.head(5)

5     180591.0
6     113103.0
14    166855.0
15    254348.0
16    164455.0
Name: total_votes, dtype: float64

In [329]:
print (type(y))
print (y.shape)

<class 'pandas.core.series.Series'>
(1274,)


In [330]:
X.shape, y.shape

((1274, 13), (1274,))

In [ ]:
from sklearn.cross_validation import train_test_splitt
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 0)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
from sklearn.linear_model import LinearRegression

# Fit regression to diabetes dataset
model = LinearRegression()
model.fit(X, y)

expected  = y
predicted = model.predict(X)

# Evaluate fit of the model
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

In [ ]:
# we import the model first

from sklearn.linear_model import LinearRegression

# im=ntantiate the model
linreg = LinearRegression()

# fit the model to the training data(Learn the coefficient)
linreg.fit(X, y)

In [ ]:
#Linear Regression
print('the Y intercept is :', linreg.intercept_)
print ('the coefficients  are :', linreg.coef_)

In [ ]:
for idx, feature_cols in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(feature_cols, linreg.coef_[idx]))

In [ ]:
for idx, feature_cols in enumerate(X_train.columns):
    print('the Y intercept is :', linreg.intercept_)

In [ ]:
RepW['party'].unique()

In [ ]:
expected  = X
predicted = y


# Evaluate fit of the model
linreg.score(X, y)

In [ ]:
from sklearn.linear_model import Perceptron

model = Perceptron()
model.fit(X, y)

expected  = y
predicted = model.predict(X)

# Evaluate fit of the model
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()
model.fit(X, y)

expected  = y
predicted = model.predict(X)

# Evaluate fit of the model
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X, y)

expected  = y
predicted = model.predict(X)

# Evaluate fit of the model
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

In [ ]:
print(y_test.shape)

In [ ]:
linreg.score(X, y)

In [ ]:
X1 = Winner['totalspend']
print (type(X1))
print (X1.shape)

In [ ]:
Y1 = Winner['total_votes']

## we could have done also Y= data_Frame.car_price

# print the first 5 rows
Y1.head(5)

In [ ]:
print (type(Y))
print (Y.shape)

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X1, Y1,random_state = 1)

In [ ]:
sns.factorplot("winner", col="party", col_wrap=3, 
                   data=Winner[Winner.party.notnull()], kind="count", size=4, aspect=.8)
plt.show()

In [ ]:
sns.pairplot(data=Gen[['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13']])
plt.show()

In [ ]:
X.head()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None. 
    """
    
    def __init__(self, columns=None):
        self.columns  = [col for col in columns] 
        self.encoders = None
    
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode. 
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns 
        
        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns 
        }
        return self

    def transform(self, data):
        """
        Uses the encoders to transform a data frame. 
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])
        
        return output

In [ ]:
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline


def model_selection(X, y, estimator):
    """
    Test various estimators.
    """ 
    y = LabelEncoder().fit_transform(y.values.ravel())
    model = Pipeline([
         ('label_encoding', EncodeCategorical(X.keys())), 
         ('one_hot_encoder', OneHotEncoder()), 
         ('estimator', estimator)
    ])

    # Instantiate the classification model and visualizer
    model.fit(X, y)  
    
    expected  = y
    predicted = model.predict(X)
    
    # Compute and return the F1 score (the harmonic mean of precision and recall)
    return (f1_score(expected, predicted))

In [ ]:
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier

In [ ]:
model_selection(X, y, LinearSVC())

In [ ]:
sns.lmplot(x='totalspend', y='total_votes', data=Winner)
sns.lmplot(x='totalspend', y='total_votes', data=RepW)
sns.lmplot(x='totalspend', y='total_votes', data=DemW)
plt.show()

In [ ]:
sns.lmplot(x='totalspend', y='total_votes', data=Loser)
sns.lmplot(x='totalspend', y='total_votes', data=DemW)
sns.lmplot(x='totalspend', y='total_votes', data=DemL)
plt.show()

In [ ]:
Winner.info()

In [ ]:
sns.violinplot(x='party', y='costvote', data=DemW, split=True, scale='count')
sns.plt.show()

In [ ]:
sns.violinplot(x='party', y='costvote', data=RepW, split=True, scale='count')
plt.show()

In [ ]:
sns.violinplot(x='winner', y='costvote', data=Gen, split=True, scale='count')
sns.plt.show()

In [ ]:
sns.pairplot(RepW[['c1','c2','c3','c5','c6','c7','c8','c9','c10','c11','c12','c13','totalspend','costvote']], hue='winner', dropna=True)
plt.show()

In [ ]:
Winner['party'].unique()

In [ ]:
Gen.loc[(df['party'] == some_value) & df['other_column'].isin(some_values)]